In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from celluloid import Camera


# Sources
* Arms imports data: [Kaggle](https://www.kaggle.com/datasets/justin2028/arms-imports-per-country)
* Countries and continents: [Kaggle](https://www.kaggle.com/datasets/fernandol/countries-of-the-world)
